## A intenção do projeto é criar um chatbot baseado em reviews de filmes para que se possa fazer perguntas e manter uma conversa livre sobre este tema

- link do banco de dados https://www.kaggle.com/Cornell-University/movie-dialog-corpus?select=movie_lines.tsv
- referências
>- https://shanebarker.com/blog/deep-learning-chatbot/
> -https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44

In [582]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import math
import random

[nltk_data] Downloading package wordnet to /home/douglas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Opening movie reviews

In [583]:
messages = pd.read_csv('./chatdata/movie_lines_normalized.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [584]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg']

In [585]:
messages.head(10)

,msg_line,user_id,movie_id,msg
0,L1045,u0,m0,They do not!
1,L1044,u2,m0,They do to!
2,L985,u0,m0,I hope so.
3,L984,u2,m0,She okay?
4,L925,u0,m0,Let's go.
5,L924,u2,m0,Wow
6,L872,u0,m0,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,No
8,L870,u0,m0,"""""""I'm kidding. You know how sometimes you jus..."
9,L869,u0,m0,Like my fear of wearing pastels?


In [586]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304713 entries, 0 to 304712
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   msg_line  304713 non-null  object
 1   user_id   304713 non-null  object
 2   movie_id  304713 non-null  object
 3   msg       304713 non-null  object
dtypes: object(4)
memory usage: 9.3+ MB


In [587]:
messages.describe()

,msg_line,user_id,movie_id,msg
count,304713,304713,304713,304713
unique,304713,9035,659,265277
top,L171298,u4525,m289,What?
freq,1,537,1530,1679


### Cleaning the msg_line of the conversations

In [588]:
#remove charactes
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [589]:
#leaving just the number of the index, so L872 changes to 872
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [590]:
#change the column type to number
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [591]:
messages = messages.sort_values(by=['msg_line_clean'])

In [592]:
#set the column as the index
messages = messages.set_index('msg_line_clean')

In [593]:
messages.head(10)

,msg_line,user_id,movie_id,msg
msg_line_clean,,,,
49,L49,u0,m0,Did you change your hair?
50,L50,u3,m0,No.
51,L51,u0,m0,You might wanna think about it
59,L59,u9,m0,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?
64,L64,u2,m0,So they tell me...


### Removing entities

In [594]:
entities = pd.read_csv('./chatdata/entity_list.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [595]:
entities.columns = ['ent', 'type']

In [596]:
entities.head()

,ent,type
0,Kinda,ORG
1,The Dallas Times Herald,ORG
2,Queen Louisa,PERSON
3,A.M,GPE
4,Cousin Hop,PERSON


In [597]:
entities['ent_len'] = [len(e) for e in entities['ent']]

In [598]:
s = entities['ent_len'].sort_values(ascending=False).index

In [599]:
entities = entities.reindex(s)

In [600]:
entities = entities.reset_index(drop=True)

In [601]:
entities.head()

,ent,type,ent_len
0,"""""""How can the Bolshevik cause gain respect am...",WORK_OF_ART,237
1,"""""""The Premier wishes to inform the Government...",WORK_OF_ART,192
2,""""""" Come Tuesday twelve a.m. bingo these like-...",WORK_OF_ART,182
3,"""""""The suggestion of the President regarding t...",WORK_OF_ART,155
4,"""""""The Management of Boyd's takes pleasure in ...",WORK_OF_ART,146


In [602]:
data = messages['msg']

In [603]:
ent_list =  ['PERSON', 'ORG', 'NORP', 'FAC', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
#ent_list =  ['LANGUAGE']

ent = list()
for i in range(len(entities.index)):
    if entities['type'][i] in ent_list:
        ent.append(entities['ent'][i])

In [604]:
ent = list(set(ent))
print(len(ent))
print(ent)

28514
['Karimojo Bell', 'One-Ball McCall', 'Mandy', 'JP Monroe', 'Vitamin C.', 'Bye Doc', 'Joanna Bourne', 'Trang', 'Catholic Jew Scientologist', 'Bergs', 'Holy Christ', 'Frankie Lazzaro', 'Comrade Dowd', 'VFW', 'Zerelda', 'Kal-El', 'Wendell Walker', 'Native Americans', 'Eventually Cassius', 'KEG HOUSE', "Hank Loomis'", 'Goodnight John', 'Huey Lewis', 'Granville', 'Nationwide Teletype', 'Beyond Good and Evil', 'Pbe', 'Joe Mexican', 'Shanghai', 'Boeman', 'a Pop-Pop', 'Beginner', 'BONBONS', 'Amber', "the Dons the Commission's", 'The Crow', 'the Boulevard Laundromat', 'Dore', 'Cinquecento', 'Ganz', 'Harry Powell', 'Shilts Legal Services', 'Lonely Hearts', 'Cherry Street', 'Turnbull', 'TOL', 'Miller Dream', 'Narco', 'Christy?', 'Capel', 'Temple University', 'Santa Maria', 'Anything Auggie', 'Joe Gillis', 'Columbus', 'Looseleaf Harper', 'Nietzche', 'Philby Dr.', 'Good morrow cousin', 'Goddamnit Walter', 'the Brothers Hun', 'Hotel', 'Apocalypse Now', 'Tell Graham', 'Nellis', 'Almond Joy', 'K

In [605]:
dict = {}
for n in ent:
    dict[n] = len(n)

In [606]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [607]:
dict

{'"""How can the Bolshevik cause gain respect among the Moslems if your three representatives Buljanoff Iranoff and Kopalski get so drunk that they throw a carpet out of their hotel window and complain to the management that it didn\'t fly"': 237,
 '"""The Premier wishes to inform the Government of the United States that it will be impossible for him to attend the meeting suggested by the President unless the meeting is held in Moscow."""': 192,
 '""" Come Tuesday twelve a.m. bingo these like-minded deviates log on and start yakking it up: explicit sex crime gossip who did what to whom who wants to do what when why and how."""': 182,
 '"""The suggestion of the President regarding the possibility of a meeting in Moscow would be unacceptable to Her Majesty\'s Government at the present time."': 155,
 '"""The Management of Boyd\'s takes pleasure in requesting the company of Mr. Richard Starkey that\'s you in their recently refinished gaming rooms."': 146,
 '"""Well Jim I says, it makes me 

In [622]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    corpus = [c for c in corpus if c not in list(dict.keys())]
    return ' '.join(corpus)    

In [623]:
%%time
messages['msg_pre_processed'] = [remove_entity(m) for m in list(data)]

CPU times: user 59min 18s, sys: 859 ms, total: 59min 18s
Wall time: 59min 20s


In [624]:
#chepoint
messages.to_csv('./chatdata/movie_lines_pre_processed.tsv', index=False, sep='\t', header=True)

### Opening conversation sequence

In [625]:
#read the file with the conversation sequence
conv_seq = pd.read_csv('./chatdata/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [626]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [627]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


In [628]:
conv_seq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user1_id  83097 non-null  object
 1   user2_id  83097 non-null  object
 2   movie_id  83097 non-null  object
 3   sequence  83097 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [629]:
conv_seq.describe()

,user1_id,user2_id,movie_id,sequence
count,83097,83097,83097,83097
unique,5420,5608,617,83097
top,u4331,u1475,m289,['L114180' 'L114181' 'L114182' 'L114183' 'L114...
freq,193,187,338,1


### Build conversation sequence

In [630]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [631]:
def seq_to_list(seq):
    seq_list = [remove_char(s) for s in seq]
    return seq_list

In [632]:
#initializing the msg_2 column
messages['msg_2'] = '-'

In [633]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [634]:
#link each message with its answer
for conv in conv_seq['sequence']:
    #split each sequence by space
    seq = split_conversation(conv)

    #remove the char L from the sequences
    txt_alt = [remove_char(s) for s in seq]

    #use the conversation sequence to build the target answer for each message
    link_conversations(txt_alt, messages, 'msg_pre_processed', 'msg_2')

In [635]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,you change your hair?
50,L50,u3,m0,No.,You might think about it,No.
51,L51,u0,m0,You might wanna think about it,-,You might think about it
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?,-,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?,So they tell me...,You the new guy?
64,L64,u2,m0,So they tell me...,C'mon. I'm supposed to give you the tour.,So they tell me...


## Pre processing the msg

In [636]:
data = messages['msg_pre_processed']

In [637]:
lemmatizer = WordNetLemmatizer()
def pre_processing_text(corpus):
    #remove html tags
    corpus = re.sub(r'<.*?>', '', str(corpus))
    
    #remove non-alphanumeric characters
    corpus = re.sub(r'[^a-z A-Z 0-9 \s]', '', str(corpus))
    
    #remove duplicated spaces
    corpus = re.sub(r' +', ' ', str(corpus))
    
    #capitalization
    corpus = corpus.lower()
    
    #tokenization
    corpus = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    
    #lammatization
    corpus = [lemmatizer.lemmatize(c) for c in corpus]
    
    #remove punctuation
    corpus = [t for t in corpus if t not in string.punctuation]
    
    #remove stopwords
    #it makes the model worst
    #stopwords_ = stopwords.words("english")
    #corpus = [t for t in corpus if t not in stopwords_]
    
    corpus = ' '.join(corpus)

    return corpus

In [638]:
%%time
data_pre_processed = [pre_processing_text(str(m)) for m in data]
data_pre_processed

CPU times: user 18.4 s, sys: 1.12 s, total: 19.5 s
Wall time: 19.5 s


['you change your hair',
 'no',
 'you might think about it',
 'i missed you',
 'it say here you exposed yourself to a group of freshman girl',
 'it wa a bratwurst i wa eating lunch',
 'with the teeth of your zipper',
 'you the new guy',
 'so they tell me',
 'cmon im supposed to give you the tour',
 'so which you from',
 'actually howd you',
 'i wa kidding people actually live there',
 'yeah a couple outnumbered by the cow though',
 'how many people were in your old school',
 'thirtytwo',
 'get out',
 'how many people go here',
 'couple thousand most of them evil',
 'that im used to',
 'yeah but these guy have never seen a horse they just off to eastwood',
 'that girl i',
 'you burn you pine you perish',
 'who is she',
 'stratford sophomore even think about it',
 'why not',
 'i could start with your haircut but it doesnt matter shes not allowed to date until her older sister doe and thats an impossibility',
 'stratford my my youve been terrorizing m again',
 'expressing my opinion is no

In [639]:
messages['msg_pre_processed'] = data_pre_processed

### Checking for duplicated messages in msg

In [640]:
data = messages['msg_pre_processed']

In [641]:
dict = {}
for n in data:
    if n in dict:
        dict[n] = dict[n] + 1
    else:
        dict[n] = 1

In [642]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [643]:
dict

{'': 3528,
 'yes': 1821,
 'no': 1612,
 'yeah': 1486,
 'why': 612,
 'okay': 477,
 'i dont know': 463,
 'what': 363,
 'thank you': 336,
 'oh': 326,
 'who': 314,
 'hello': 311,
 'right': 293,
 'yes sir': 281,
 'why not': 273,
 'what is it': 268,
 'really': 265,
 'i know': 255,
 'hi': 255,
 'what do you mean': 241,
 'thanks': 221,
 'im sorry': 217,
 'what are you doing': 209,
 'well': 203,
 'whats that': 201,
 'nothing': 193,
 'you': 193,
 'uhhuh': 191,
 'thats right': 185,
 'what happened': 181,
 'shit': 180,
 'where': 175,
 'of course': 174,
 'what are you talking about': 174,
 'hey': 164,
 'me': 153,
 'how': 144,
 'excuse me': 142,
 'sorry': 136,
 'sir': 136,
 'up': 135,
 'so': 135,
 'please': 133,
 'jesus': 132,
 'come on': 124,
 'what do you want': 119,
 'all right': 116,
 'whats wrong': 109,
 'it': 109,
 'here': 106,
 'are you going': 99,
 'what are you doing here': 98,
 'i': 94,
 'when': 93,
 'i see': 93,
 'what do you think': 92,
 'no sir': 91,
 'ok': 89,
 'great': 87,
 'who are yo

In [644]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'did you change your hair']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,


In [645]:
#get the repeated messages
d_list = list()
for k in dict:
    if dict[k] > 1:
        d_list.append(k)

In [646]:
d_list

['',
 'yes',
 'no',
 'yeah',
 'why',
 'okay',
 'i dont know',
 'what',
 'thank you',
 'oh',
 'who',
 'hello',
 'right',
 'yes sir',
 'why not',
 'what is it',
 'really',
 'i know',
 'hi',
 'what do you mean',
 'thanks',
 'im sorry',
 'what are you doing',
 'well',
 'whats that',
 'nothing',
 'you',
 'uhhuh',
 'thats right',
 'what happened',
 'shit',
 'where',
 'of course',
 'what are you talking about',
 'hey',
 'me',
 'how',
 'excuse me',
 'sorry',
 'sir',
 'up',
 'so',
 'please',
 'jesus',
 'come on',
 'what do you want',
 'all right',
 'whats wrong',
 'it',
 'here',
 'are you going',
 'what are you doing here',
 'i',
 'when',
 'i see',
 'what do you think',
 'no sir',
 'ok',
 'great',
 'who are you',
 'and',
 'but',
 'whats the matter',
 'now',
 'what for',
 'i dont think so',
 'i cant',
 'you okay',
 'maybe',
 'like what',
 'whats this',
 'i dont understand',
 'my god',
 'whats going on',
 'sure',
 'dad',
 'about what',
 'let go',
 'for what',
 'no thanks',
 'are you sure',
 'i lo

In [647]:
messages = messages.drop_duplicates(subset=['msg_pre_processed'])

In [648]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'did you change your hair']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,


In [649]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252596 entries, 49 to 666576
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   msg_line           252596 non-null  object
 1   user_id            252596 non-null  object
 2   movie_id           252596 non-null  object
 3   msg                252596 non-null  object
 4   msg_2              252596 non-null  object
 5   msg_pre_processed  252596 non-null  object
dtypes: object(6)
memory usage: 13.5+ MB


In [650]:
messages.describe()

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
count,252596,252596,252596,252596,252596,252596
unique,252596,9000,617,252596,159732,252596
top,L171298,u3681,m289,Remember the Alamo?,-,a forlorn hope but there is a chance
freq,1,434,1240,1,69665,1


### Removing nan msg origined by '' messages

In [651]:
#filling the nan messages with a string- not necessary
#messages = messages.fillna('UNKNOWN')

### Removing apostrophes (need for embedding)

In [652]:
#The quality of the model became worst
#messages['msg_pre_processed'] = [ word.replace("\'","") for word in messages['msg_pre_processed']]

### Filling '-' messages with a generic one

In [653]:
#return generic answer
def generic_answer(txt):
  asw_list = ['talk more about it',
              'can you explain it better?',
              'I need to think more about it',
              'maybe...'
              ]
  if txt == '-':
    return random.choice(asw_list)
  return txt

In [654]:
#seting a generic answer to the messages without answer
messages['msg_2'] = [generic_answer(msg) for msg in messages['msg_2']]

<ipython-input-654-416b64f870a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['msg_2'] = [generic_answer(msg) for msg in messages['msg_2']]


In [655]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,you change your hair
50,L50,u3,m0,No.,You might think about it,no
51,L51,u0,m0,You might wanna think about it,can you explain it better?,you might think about it
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,i missed you
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,it say here you exposed yourself to a group of...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?,it wa a bratwurst i wa eating lunch
62,L62,u8,m0,With the teeth of your zipper?,I need to think more about it,with the teeth of your zipper
63,L63,u7,m0,You the new guy?,So they tell me...,you the new guy
64,L64,u2,m0,So they tell me...,C'mon. I'm supposed to give you the tour.,so they tell me


### Tagging the msg with classes

In [656]:
def define_target(corpus):
    
    if '?' in corpus:
        return 1
    else:
        return 0

In [657]:
data = messages['msg']

In [658]:
messages['target'] = [define_target(m) for m in data]

<ipython-input-658-8c498f5d1460>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['target'] = [define_target(m) for m in data]


In [659]:
messages['target'] = messages['target'].astype(int)

<ipython-input-659-359a35de7aa5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['target'] = messages['target'].astype(int)


In [660]:
messages.head(20)

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,you change your hair,1
50,L50,u3,m0,No.,You might think about it,no,0
51,L51,u0,m0,You might wanna think about it,can you explain it better?,you might think about it,0
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,i missed you,0
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,it say here you exposed yourself to a group of...,0
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?,it wa a bratwurst i wa eating lunch,0
62,L62,u8,m0,With the teeth of your zipper?,I need to think more about it,with the teeth of your zipper,1
63,L63,u7,m0,You the new guy?,So they tell me...,you the new guy,1
64,L64,u2,m0,So they tell me...,C'mon. I'm supposed to give you the tour.,so they tell me,0


### Save data

In [661]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252596 entries, 49 to 666576
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   msg_line           252596 non-null  object
 1   user_id            252596 non-null  object
 2   movie_id           252596 non-null  object
 3   msg                252596 non-null  object
 4   msg_2              252596 non-null  object
 5   msg_pre_processed  252596 non-null  object
 6   target             252596 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 15.4+ MB


In [662]:
messages

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,you change your hair,1
50,L50,u3,m0,No.,You might think about it,no,0
51,L51,u0,m0,You might wanna think about it,can you explain it better?,you might think about it,0
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,i missed you,0
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,it say here you exposed yourself to a group of...,0
...,...,...,...,...,...,...,...
666522,L666522,u9034,m616,So far only their scouts. But we have had repo...,can you explain it better?,so far only their scout but we have had report...,0
666546,L666546,u9027,m616,Splendid site Crealock splendil I want to esta...,Certainly Sin,splendid site splendil i want to establish cam...,0
666547,L666547,u9029,m616,Certainly Sin,I need to think more about it,certainly sin,0


In [663]:
messages.to_csv('./chatdata/movie_lines_pre_processed.tsv', index=False, sep='\t', header=True)